# MLRun Job Example

### Setup MLRun

In [1]:
import os
from os import path
from mlrun import set_environment, new_project, mlconf, code_to_function, mount_v3io, get_run_db

# Set the default environment configuration
project_name, artifact_path = set_environment(
    project="igz-onboarding",
    artifact_path="/User/artifacts/{{run.project}}/{{workflow.uid}}"
)

# Create project
project_path = path.abspath("project")
project = new_project(name=project_name, context=project_path)

print(f"Project name: {project_name}\nProject path: {project_path}")
print(f"Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}")

Project name: igz-onboarding
Project path: /User/igz_repos/igz-onboarding/project
Artifacts path: /User/artifacts/{{run.project}}/{{workflow.uid}}
MLRun DB path: http://mlrun-api:8080


### Build Docker Image

In [2]:
image = f"docker-registry.{os.getenv('IGZ_NAMESPACE_DOMAIN')}:80/new-image"
image

'docker-registry.default-tenant.app.us-sales-fs.iguazio-cd0.com:80/new-image'

In [3]:
# # Build Docker Image (only needs to be run once)
# build_image = new_function(name="build-image", kind="job")
# build_image.build_config(
#     image=image, base_image="mlrun/ml-models-gpu", commands=["pip install v3io_frames"]
# )
# build_image.deploy(with_mlrun=False)

### Define MLRun Function

In [4]:
# Create MLRun function from Python file
fn = code_to_function(
    name="training-demo",
    project=project_name,
    filename="src_code/mlrun_fn.py",
    handler="handler",
    kind='job',
    image="mlrun/mlrun"
)

### Run MLRun Function Locally

In [5]:
run = fn.run(params={'name' : "testing", "value" : 123}, artifact_path=artifact_path, local=True)

> 2021-05-13 14:36:03,126 [info] starting run training-demo-handler uid=fc81cbc5ada5485c912a137c46d72012 DB=http://mlrun-api:8080
> 2021-05-13 14:36:03,300 [info] *****Hello from MLRun, testing!*****


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
igz-onboarding,...46d72012,0,May 13 14:36:03,completed,training-demo-handler,v3io_user=nickkind=owner=nickhost=nick-jupyter-8b7d7474c-sdlr6,,name=testingvalue=123,value=123,


to track results use .show() or .logs() or in CLI: 
!mlrun get run fc81cbc5ada5485c912a137c46d72012 --project igz-onboarding , !mlrun logs fc81cbc5ada5485c912a137c46d72012 --project igz-onboarding
> 2021-05-13 14:36:03,397 [info] run executed, status=completed


### Run MLRun Function on Cluster

In [6]:
run = fn.run(params={'name' : "testing", "value" : 123}, artifact_path=artifact_path)

> 2021-05-13 14:36:03,402 [info] starting run training-demo-handler uid=26e150cb7e244ab0bc83ef2fc069a1ee DB=http://mlrun-api:8080
> 2021-05-13 14:36:03,571 [info] Job is running in the background, pod: training-demo-handler-wszs4
> 2021-05-13 14:36:06,729 [info] *****Hello from MLRun, testing!*****
> 2021-05-13 14:36:06,765 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
igz-onboarding,...c069a1ee,0,May 13 14:36:06,completed,training-demo-handler,v3io_user=nickkind=jobowner=nickhost=training-demo-handler-wszs4,,name=testingvalue=123,value=123,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 26e150cb7e244ab0bc83ef2fc069a1ee --project igz-onboarding , !mlrun logs 26e150cb7e244ab0bc83ef2fc069a1ee --project igz-onboarding
> 2021-05-13 14:36:09,698 [info] run executed, status=completed


### View Run Logs

In [7]:
db = get_run_db()
resp = db.watch_log(run.uid(), project=project_name)

> 2021-05-13 14:36:06,729 [info] *****Hello from MLRun, testing!*****
> 2021-05-13 14:36:06,765 [info] run executed, status=completed

